In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [2]:
upper_bounds = [0.99, 0.99, 0.99, 0.99, 0.25]
lower_bounds = [0.01, 0.001, 0.01, 0.6, 0.05]

# Pair up the corresponding bounds and calculate their average
averages = [(u + l) / 2 for u, l in zip(upper_bounds, lower_bounds)]
avg_Cm, avg_Ch, avg_Bm, avg_Bh, avg_T = averages

headers = "Cm,Ch,Bm,Bh,T,sR,sG,sB"
lut_path = r"D:\Github\Deep-Albedo\cpp_monte_carlo\lut_rgb.csv"

df = pd.read_csv(lut_path, names=headers.split(","), header=None)
#remove row 0
df = df.iloc[1:]
print(df.head())

#print length of df
print(len(df))
# convert all columns to float
for column in df.columns:
    df[column] = pd.to_numeric(df[column], errors='coerce')

#convert sR, sG, sB to int
rounded_df = df.round({'sR': 0, 'sG': 0, 'sB': 0})
# Step 1: Find duplicate RGB values
duplicates = rounded_df[rounded_df.duplicated(subset=['sR', 'sG', 'sB'], keep=False)].copy() # added .copy()
#print all duplicates
print(duplicates.head())

# Step 2: Calculate the 'likelihood' score for each row
duplicates['likelihood'] = (abs(duplicates['Cm'] - avg_Cm) +
                            abs(duplicates['Ch'] - avg_Ch) +
                            abs(duplicates['Bm'] - avg_Bm) +
                            abs(duplicates['Bh'] - avg_Bh) +
                            abs(duplicates['T'] - avg_T))

# Step 3: Sort by RGB values and likelihood, keeping the row with the lowest likelihood for each RGB group
most_likely_duplicates = duplicates.sort_values(['sR', 'sG', 'sB', 'likelihood']).drop_duplicates(subset=['sR', 'sG', 'sB'])

# Now, most_likely_duplicates should contain your desired rows

# First, remove all duplicates from the original dataframe
df_no_duplicates = df.drop_duplicates(subset=['sR', 'sG', 'sB'], keep=False)

# Concatenate df_no_duplicates with most_likely_duplicates to get the final dataframe
df = pd.concat([df_no_duplicates, most_likely_duplicates])

# If you want to sort it based on index
df.sort_index(inplace=True)
df.head()
#remove duplicates

x = df[['sR', 'sG', 'sB']].to_numpy(dtype='float32')
y = df[['Cm', 'Ch', 'Bm', 'Bh', 'T']].to_numpy(dtype='float32')
#create new csv with headers
# df.to_csv(r'LUTs\large_no_duplicates.csv', index=False, header=True)


#train nn on x,y
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=True)
#remove any header values
x_train = x_train[1:]
x_test = x_test[1:]
y_train = y_train[1:]
y_test = y_test[1:]

#numpy arrays
x_train = np.asarray(x_train).reshape(-1,3).astype('float32')
x_test = np.asarray(x_test).reshape(-1,3).astype('float32')
print(f"length of df {len(df)}")
print(f"bef norm x_train[0] {x_train[0]}")

#normalize
x_train = x_train/255.0
x_test = x_test/255.0
print(f"aft norm x_train[0] {x_train[0]}")

print(f"length of x_train {len(x_train)}")
print(f"length of x_test {len(x_test)}")
print(f"length of y_train {len(y_train)}")
print(f"length of y_test {len(y_test)}")
df.head()
print(f"length of df {len(df)}")
#print random 3 rows
#print unique values of Cm,Ch,Bm,Bh,T
# print(f"unique Cm {df['Cm'].unique()}")
# print(f"unique Ch {df['Ch'].unique()}")
# print(f"unique Bm {df['Bm'].unique()}")
# print(f"unique Bh {df['Bh'].unique()}")
# print(f"unique T {df['T'].unique()}")
#as sorted lists
C_m = sorted(df['Cm'].unique())
C_h = sorted(df['Ch'].unique())
B_m = sorted(df['Bm'].unique())
B_h = sorted(df['Bh'].unique())
T = sorted(df['T'].unique())
print(f"Cm = {C_m}")
print(f"Ch = {C_h}")
print(f"Bm = {B_m}")
print(f"Bh = {B_h}")
print(f"T = {T}")
#min max for each
min_vals = [min(C_m), min(C_h), min(B_m), min(B_h), min(T)]
max_vals = [max(C_m), max(C_h), max(B_m), max(B_h), max(T)]
print(f"upper bounds = {max_vals}")
print(f"lower bounds = {min_vals}")
#integer arrays for sR,sG,sB 0 to 255
# Assuming df is your DataFrame and it has columns 'sR', 'sG', 'sB'
df[['sR', 'sG', 'sB']] = df[['sR', 'sG', 'sB']].astype(float)
df[['sR', 'sG', 'sB']] = df[['sR', 'sG', 'sB']].applymap(np.round)
# Add a 'count' column that counts the number of identical RGB values
df['count'] = df.groupby(['sR', 'sG', 'sB'])['sR'].transform('count')
print(f"number of repeated RGB values {len(df[df['count'] > 1])}")

C:\Users\Aai\AppData\Local\Temp\ipykernel_23332\3161401370.py:11: DtypeWarning: Columns (0,1,2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(lut_path, names=headers.split(","), header=None)


      Cm          Ch Bm Bh          T       sR       sG       sB
1  0.001  0.00760226  0  0       0.01  250.349  201.031   164.79
2  0.001  0.00760226  0  0  0.0271429  251.037  202.084  167.913
3  0.001  0.00760226  0  0  0.0785714  251.832  203.648   172.33
4  0.001  0.00760226  0  0  0.0614286  251.309  202.753  170.808
5  0.001  0.00760226  0  0  0.0957143  252.102  204.416  174.004
749373
      Cm        Ch   Bm   Bh         T     sR     sG     sB
1  0.001  0.007602  0.0  0.0  0.010000  250.0  201.0  165.0
2  0.001  0.007602  0.0  0.0  0.027143  251.0  202.0  168.0
3  0.001  0.007602  0.0  0.0  0.078571  252.0  204.0  172.0
4  0.001  0.007602  0.0  0.0  0.061429  251.0  203.0  171.0
5  0.001  0.007602  0.0  0.0  0.095714  252.0  204.0  174.0
length of df 876726
bef norm x_train[0] [99.108 48.952 39.707]
aft norm x_train[0] [0.38865882 0.19196862 0.15571372]
length of x_train 701379
length of x_test 175345
length of y_train 701379
length of y_test 175345
length of df 876726
Cm = [n

C:\Users\Aai\AppData\Local\Temp\ipykernel_23332\3161401370.py:109: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['sR', 'sG', 'sB']] = df[['sR', 'sG', 'sB']].applymap(np.round)


number of repeated RGB values 780784


In [3]:
# -----------------------------
# Reproducibility (np.random.seed(7) equivalent)
# -----------------------------
np.random.seed(7)
torch.manual_seed(7)
torch.cuda.manual_seed_all(7)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# -----------------------------
# Hyperparameters (from your code)
# -----------------------------
BATCH_SIZE = 4096 * 16
NUM_NEURONS = 75
NUM_LAYERS = 2
NUM_EPOCHS = 200
LR = 1e-4
MLR = 1e-6  # unused in your compile; keeping for parity

In [4]:
import torch
from torch.utils.data import Dataset, DataLoader

class AEDataset(Dataset):
    def __init__(self, x, y):
        """
        x: [N,3]  (albedo / RGB in your naming)
        y: [N,5]  (parameters / latent in your naming)
        """
        self.x = torch.as_tensor(x, dtype=torch.float32)
        self.y = torch.as_tensor(y, dtype=torch.float32)

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        x_i = self.x[idx]   # [3]
        y_i = self.y[idx]   # [5]

        # inputs (match Keras x=[x_train, y_train, x_train])
        enc_in = x_i
        dec_in = y_i
        end_in = x_i

        # targets (match Keras y=[y_train, x_train, x_train])
        enc_true = y_i
        dec_true = x_i
        end_true = x_i

        return enc_in, dec_in, end_in, enc_true, dec_true, end_true


train_loader = DataLoader(
    AEDataset(x_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False
)

val_loader = DataLoader(
    AEDataset(x_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False,
    drop_last=False
)


In [7]:
import os
import time
from datetime import datetime

import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import json


# -----------------------------
# DEVICE
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# -----------------------------
# MODEL COMPONENTS
# -----------------------------
class Encoder(nn.Module):
    def __init__(self, in_dim=3, hidden_dim=NUM_NEURONS, num_layers=NUM_LAYERS, out_dim=5):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(in_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        self.mlp = nn.Sequential(*layers)
        self.out = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        x = self.mlp(x)
        return self.out(x)


class Decoder(nn.Module):
    def __init__(self, in_dim=5, hidden_dim=NUM_NEURONS, num_layers=NUM_LAYERS, out_dim=3):
        super().__init__()
        layers = []
        for i in range(num_layers):
            layers.append(nn.Linear(in_dim if i == 0 else hidden_dim, hidden_dim))
            layers.append(nn.ReLU())
        self.mlp = nn.Sequential(*layers)
        self.out = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        x = self.mlp(x)
        return self.out(x)


class AutoEncoder(nn.Module):
    def __init__(self, encoder: nn.Module, decoder: nn.Module):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, encoder_in, decoder_in, end_to_end_in):
        enc_out = self.encoder(encoder_in)
        dec_out = self.decoder(decoder_in)
        end_out = self.decoder(self.encoder(end_to_end_in))
        return enc_out, dec_out, end_out





# -----------------------------
# Loss functions (same as before)
# -----------------------------
def albedo_loss(y_true, y_pred):
    return torch.sum(torch.abs(y_pred - y_true), dim=-1)

def parameter_loss(y_true, y_pred):
    return torch.sqrt(torch.sum((y_pred - y_true) ** 2, dim=-1) + 1e-12)

def end_to_end_loss(y_true, y_pred):
    return torch.sum(torch.abs(y_pred - y_true), dim=-1)

def reduce_loss(loss_per_sample, reduction="mean"):
    if reduction == "mean":
        return loss_per_sample.mean()
    elif reduction == "sum":
        return loss_per_sample.sum()
    else:
        raise ValueError("reduction must be 'mean' or 'sum'")


# -----------------------------
# Run folder creation (date/time)
# -----------------------------
def create_run_folder(base_dir="checkpoints"):
    now = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_dir = os.path.join(base_dir, now)
    os.makedirs(run_dir, exist_ok=True)
    return run_dir


# -----------------------------
# Evaluation loop (val)
# -----------------------------
def evaluate(model, dataloader, loss_weights=(0.3, 0.1, 0.6)):
    model.eval()

    total_loss_sum = 0.0
    loss1_sum = 0.0
    loss2_sum = 0.0
    loss3_sum = 0.0
    n_batches = 0

    with torch.no_grad():
        for batch in dataloader:
            enc_in, dec_in, end_in, enc_true, dec_true, end_true = batch

            enc_in   = enc_in.to(device)
            dec_in   = dec_in.to(device)
            end_in   = end_in.to(device)
            enc_true = enc_true.to(device)
            dec_true = dec_true.to(device)
            end_true = end_true.to(device)

            enc_pred, dec_pred, end_pred = model(enc_in, dec_in, end_in)

            loss1 = reduce_loss(parameter_loss(enc_true, enc_pred), "mean")
            loss2 = reduce_loss(albedo_loss(dec_true, dec_pred), "mean")
            loss3 = reduce_loss(end_to_end_loss(end_true, end_pred), "mean")

            total_loss = loss_weights[0]*loss1 + loss_weights[1]*loss2 + loss_weights[2]*loss3

            total_loss_sum += total_loss.item()
            loss1_sum += loss1.item()
            loss2_sum += loss2.item()
            loss3_sum += loss3.item()
            n_batches += 1

    if n_batches == 0:
        return {"total": 0.0, "loss1": 0.0, "loss2": 0.0, "loss3": 0.0}

    return {
        "total": total_loss_sum / n_batches,
        "loss1": loss1_sum / n_batches,
        "loss2": loss2_sum / n_batches,
        "loss3": loss3_sum / n_batches,
    }


# -----------------------------
# Training loop (train + val)
# -----------------------------
def train(
    model,
    train_loader,
    val_loader,
    optimizer,
    scheduler,
    num_epochs=200,
    loss_weights=(0.3, 0.1, 0.6),
    base_ckpt_dir="checkpoints",
    checkpoint_period=200,
    print_period=25,
    save_json_each_epoch=True,
):
    run_dir = create_run_folder(base_ckpt_dir)
    best_ckpt_path = os.path.join(run_dir, "best.pt")
    last_ckpt_path = os.path.join(run_dir, "last.pt")
    json_path = os.path.join(run_dir, "history.json")

    print("Using device:", device)
    print("Run folder:", run_dir)

    best_train_loss = float("inf")

    history = {
        "config": {
            "num_epochs": num_epochs,
            "loss_weights": loss_weights,
            "optimizer": optimizer.__class__.__name__,
            "scheduler": scheduler.__class__.__name__,
            "device": str(device),
        },
        "epochs": []
    }

    for epoch in range(num_epochs):
        model.train()

        total_loss_sum = 0.0
        loss1_sum = 0.0
        loss2_sum = 0.0
        loss3_sum = 0.0
        n_batches = 0

        for batch in train_loader:
            enc_in, dec_in, end_in, enc_true, dec_true, end_true = batch

            enc_in   = enc_in.to(device)
            dec_in   = dec_in.to(device)
            end_in   = end_in.to(device)
            enc_true = enc_true.to(device)
            dec_true = dec_true.to(device)
            end_true = end_true.to(device)

            optimizer.zero_grad()

            enc_pred, dec_pred, end_pred = model(enc_in, dec_in, end_in)

            loss1 = reduce_loss(parameter_loss(enc_true, enc_pred), "mean")
            loss2 = reduce_loss(albedo_loss(dec_true, dec_pred), "mean")
            loss3 = reduce_loss(end_to_end_loss(end_true, end_pred), "mean")

            total_loss = loss_weights[0]*loss1 + loss_weights[1]*loss2 + loss_weights[2]*loss3

            total_loss.backward()
            optimizer.step()

            total_loss_sum += total_loss.item()
            loss1_sum += loss1.item()
            loss2_sum += loss2.item()
            loss3_sum += loss3.item()
            n_batches += 1

        train_stats = {
            "total": total_loss_sum / max(n_batches, 1),
            "loss1": loss1_sum / max(n_batches, 1),
            "loss2": loss2_sum / max(n_batches, 1),
            "loss3": loss3_sum / max(n_batches, 1),
        }

        val_stats = evaluate(model, val_loader, loss_weights=loss_weights)

        # Keras ReduceLROnPlateau monitors train loss in your code
        scheduler.step(train_stats["total"])
        current_lr = optimizer.param_groups[0]["lr"]

        # Logging record for this epoch
        epoch_record = {
            "epoch": epoch,
            "lr": current_lr,
            "train": train_stats,
            "val": val_stats,
        }
        history["epochs"].append(epoch_record)

        # Save JSON file continuously (optional)
        if save_json_each_epoch:
            with open(json_path, "w") as f:
                json.dump(history, f, indent=4)

        # Print callback every N epochs
        if epoch % print_period == 0:
            print(
                f"epoch: {epoch} | "
                f"train_total={train_stats['total']:.6f} "
                f"(L1={train_stats['loss2']:.6f}, L2={train_stats['loss1']:.6f}, end={train_stats['loss3']:.6f}) | "
                f"val_total={val_stats['total']:.6f} "
                f"(L1={val_stats['loss2']:.6f}, L2={val_stats['loss1']:.6f}, end={val_stats['loss3']:.6f}) | "
                f"lr={current_lr:.2e}"
            )

        # Save BEST model (like ModelCheckpoint(save_best_only=True))
        if train_stats["total"] < best_train_loss:
            best_train_loss = train_stats["total"]
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "best_train_loss": best_train_loss,
                    "history_path": json_path,
                },
                best_ckpt_path,
            )

        # Save periodic checkpoint (every checkpoint_period epochs)
        if (epoch + 1) % checkpoint_period == 0:
            torch.save(
                {
                    "epoch": epoch,
                    "model_state_dict": model.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "train_total_loss": train_stats["total"],
                    "val_total_loss": val_stats["total"],
                    "history_path": json_path,
                },
                last_ckpt_path,
            )

    # final save JSON (ensures up-to-date)
    with open(json_path, "w") as f:
        json.dump(history, f, indent=4)

    print("\nTraining complete.")
    print("Best checkpoint saved at:", best_ckpt_path)
    print("History saved at:", json_path)

    return history, run_dir


In [8]:
encoder_net = Encoder().to(device)
decoder_net = Decoder().to(device)
model = AutoEncoder(encoder_net, decoder_net).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

scheduler = ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.01,
    patience=5,
    threshold=1e-4,
    cooldown=0,
    min_lr=MLR,
)

history, run_dir = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    scheduler=scheduler,
    num_epochs=NUM_EPOCHS,
    loss_weights=(0.3, 0.1, 0.6),
    base_ckpt_dir="checkpoints",
    checkpoint_period=200,
    print_period=25,
    save_json_each_epoch=True,
)


print("Best checkpoint saved at:", best_path)
print("Run directory:", run_dir)


Using device: cuda
Run folder: checkpoints\2025-12-31_18-54-39
epoch: 0 | train_total=0.998119 (L1=1.056506, L2=0.903351, end=1.035772) | val_total=0.969549 (L1=1.020336, L2=0.883473, end=1.004122) | lr=1.00e-04
epoch: 25 | train_total=0.277744 (L1=0.392116, L2=0.602354, end=0.096377) | val_total=0.276219 (L1=0.389906, L2=0.600194, end=0.095284) | lr=1.00e-04
epoch: 50 | train_total=0.234435 (L1=0.294518, L2=0.540593, end=0.071342) | val_total=0.233522 (L1=0.291879, L2=0.539600, end=0.070758) | lr=1.00e-04
epoch: 75 | train_total=0.195861 (L1=0.238998, L2=0.512280, end=0.030462) | val_total=0.195520 (L1=0.236697, L2=0.511698, end=0.030568) | lr=1.00e-04
epoch: 100 | train_total=0.185279 (L1=0.208021, L2=0.502692, end=0.022783) | val_total=0.184924 (L1=0.206521, L2=0.502277, end=0.022648) | lr=1.00e-04
epoch: 125 | train_total=0.178411 (L1=0.192968, L2=0.496571, end=0.016905) | val_total=0.177970 (L1=0.191745, L2=0.496285, end=0.016516) | lr=1.00e-04
epoch: 150 | train_total=0.173049 (L

NameError: name 'best_path' is not defined

In [9]:
import numpy as np
import torch
import time
import math

# device setup (equivalent to tf.device('/device:GPU:0'))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Make sure encoder / decoder are already defined and moved to device:
# encoder.to(device)
# decoder.to(device)
# encoder.eval()
# decoder.eval()


def encode(image, encoder):
    """
    PyTorch version of TensorFlow encode()
    Input: image (numpy array), encoder (torch.nn.Module)
    Output: pred_maps (numpy), elapsed (float), (WIDTH, HEIGHT)
    """
    print(f"Image shape at start of encoder method: {image.shape}")

    # Handle 2D flattened input vs image input
    if len(image.shape) == 2:
        WIDTH = HEIGHT = int(math.sqrt(image.shape[0]))
        # your code: reshape(-1,4) then later reshape(W*H,3)
        # This implies the 4th channel was ignored later.
        # We'll keep it identical:
        image = np.asarray(image).reshape(-1, 4).astype("float32")
    else:
        WIDTH = image.shape[0]
        HEIGHT = image.shape[1]
        image = np.asarray(image).astype("float32")

    # match your Keras reshape
    image = image.reshape(WIDTH * HEIGHT, 3)

    start = time.time()
    print(f"Image shape before encoder inference: {image.shape}")

    # Convert to torch tensor and run model inference
    x = torch.from_numpy(image).to(device)

    encoder.eval()
    with torch.no_grad():
        pred_maps = encoder(x)

    # Convert back to numpy
    pred_maps = pred_maps.detach().cpu().numpy()

    end = time.time()
    elapsed = end - start

    # reshape output to (WIDTH*HEIGHT, 5)
    pred_maps = pred_maps.reshape(WIDTH * HEIGHT, 5)

    return pred_maps, elapsed, (WIDTH, HEIGHT)


def decode(encoded, decoder):
    """
    PyTorch version of TensorFlow decode()
    Input: encoded (numpy array), decoder (torch.nn.Module)
    Output: recovered (numpy), elapsed (float), (WIDTH, HEIGHT)
    """
    print(f"Image shape going into encoder: {encoded.shape}")

    start = time.time()

    # Handle 2D flattened input vs (W,H,C) style input
    if len(encoded.shape) == 2:
        WIDTH = HEIGHT = int(math.sqrt(encoded.shape[0]))
        encoded = np.asarray(encoded).reshape(-1, 5).astype("float32")
    else:
        WIDTH = encoded.shape[0]
        HEIGHT = encoded.shape[1]
        encoded = np.asarray(encoded).astype("float32")

    print(f"encoded shape going into decoder: {encoded.shape}")

    # Torch inference
    x = torch.from_numpy(encoded).to(device)

    decoder.eval()
    with torch.no_grad():
        recovered = decoder(x)

    recovered = recovered.detach().cpu().numpy()

    end = time.time()
    elapsed = end - start

    # reshape output to RGB image
    recovered = recovered.reshape(WIDTH, HEIGHT, 3)

    return recovered, elapsed, (WIDTH, HEIGHT)
